In [55]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv("steam_games.csv")

In [3]:
df.head(5)

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


# Data Preparation

The data preparation I did is to prepare the features that have a potential to be an identifier of similarity between games. Basically these features contains information about the game such as description, rating, genre, and any information we can get. In this project, I used features like description, genre, rating, and another details about the game.

## Types Games

In [4]:
df["types"].unique()

array(['app', 'bundle', 'sub', nan], dtype=object)

I am curious about this feature. Turns out, there are three different types of games in Steam. But, there is also nan type or null data.

In [5]:
df[(df["types"]!='app')& (df["types"]!='bundle')&(df["types"]!='sub')]

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
704,https://store.steampowered.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$29.99,$23.99
35169,https://store.steampowered.com/digitalgiftcards/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For the games that have no type, these games also did not have many information about theirself. Index 704 and 35169 does not have enough info to be analyzed, because most of the data of these rows are Null. So I drop it.

In [6]:
#drop types=nan
df = df.drop([704,35169])
df = df.reset_index()

## Description

There are two columns that can be used as description games feature, these are `desc_snippet` and `game_description`. I need to compare between these two columns.

In [7]:
df[["desc_snippet","game_description"]]

,desc_snippet,game_description
0,Now includes all three premium DLC packs (Unto...,"About This Game Developed by id software, the..."
1,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,About This Game PLAYERUNKNOWN'S BATTLEGROUND...
2,Take command of your own mercenary outfit of '...,About This Game From original BATTLETECH/Mec...
3,The post-soviet country of Chernarus is struck...,About This Game The post-soviet country of Ch...
4,EVE Online is a community-driven spaceship MMO...,About This Game
...,...,...
40826,NaN,"About This Content Play ""Ghost Division"" by S..."
40827,NaN,"About This Content Play ""Trippin’ on a Hole i..."
40828,NaN,About This Content Quests of Doom 4: A Midni...
40829,NaN,About This Content Get equipped with the stun...


In [8]:
df["desc_snippet"][1]

"PLAYERUNKNOWN'S BATTLEGROUNDS is a battle royale shooter that pits 100 players against each other in a struggle for survival. Gather supplies and outwit your opponents to become the last person standing."

In [9]:
df["game_description"][1]

" About This Game  PLAYERUNKNOWN'S BATTLEGROUNDS is a battle royale shooter that pits 100 players against each other in a struggle for survival. Gather supplies and outwit your opponents to become the last person standing. PLAYERUNKNOWN , aka Brendan Greene, is a pioneer of the battle royale genre and the creator of the battle royale game modes in the ARMA series and H1Z1: King of the Kill. At PUBG Corp., Greene is working with a veteran team of developers to make PUBG into the world's premiere battle royale experience."

In [10]:
df[(df["game_description"].isna())&(df["desc_snippet"].notna())]

,index,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,...,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
5,5,https://store.steampowered.com/bundle/5699/Gra...,bundle,Grand Theft Auto V: Premium Online Edition,Grand Theft Auto V: Premium Online Edition bundle,NaN,NaN,NaN,Rockstar North,Rockstar Games,...,"Single-player,Multi-player,Downloadable Conten...","English, French, Italian, German, Spanish - Sp...",NaN,"Action,Adventure",NaN,NaN,NaN,NaN,NaN,$35.18
15,15,https://store.steampowered.com/bundle/5641/Hea...,bundle,Hearts of Iron IV: Mobilization Pack,Hearts of Iron IV: Mobilization Pack bundle,NaN,NaN,NaN,Paradox Development Studio,Paradox Interactive,...,"Single-player,Multi-player,Online Multi-Player...","English, French, German, Polish, Portuguese - ...",NaN,"Simulation,Strategy",NaN,NaN,NaN,NaN,NaN,$94.45
25,25,https://store.steampowered.com/bundle/234/Port...,bundle,Portal Bundle,Portal Bundle bundle,NaN,NaN,NaN,Valve,Valve,...,"Single-player,Co-op,Steam Achievements,Full co...","English, French, German, Russian, Danish, Dutc...",NaN,"Action,Adventure",NaN,NaN,NaN,NaN,NaN,$14.98
39,39,https://store.steampowered.com/sub/18397/,sub,Doom Classic Complete,Doom Classic Complete bundle,NaN,NaN,NaN,id Software,id Software,...,"Single-player,Multi-player,Steam Cloud",English,NaN,Action,NaN,NaN,NaN,NaN,$14.99,$7.49
44,44,https://store.steampowered.com/bundle/2405/The...,bundle,The Binding of Isaac: Rebirth Complete Bundle,The Binding of Isaac: Rebirth Complete Bundle ...,NaN,NaN,NaN,"Nicalis, Inc.","Nicalis, Inc.",...,"Single-player,Multi-player,Local Multi-Player,...",English,NaN,"Action,Adventure,Indie",NaN,NaN,NaN,NaN,NaN,$35.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39363,39365,https://store.steampowered.com/bundle/8092/201...,bundle,2018 Yamaha UTV Bundle,2018 Yamaha UTV Bundle bundle,NaN,NaN,NaN,Rainbow Studios,THQ Nordic,...,"Single-player,Multi-player,Online Multi-Player...","English, French, Italian, German, Spanish - Sp...",NaN,"Racing,Simulation,Sports",NaN,NaN,NaN,NaN,NaN,$6.08
39364,39366,https://store.steampowered.com/bundle/4380/Blu...,bundle,Blue Reflection - Vacation Style Complete Set,Blue Reflection - Vacation Style Complete Set ...,NaN,NaN,NaN,"KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.",...,"Single-player,Downloadable Content,Steam Achie...","English, Japanese, Traditional Chinese",NaN,RPG,NaN,NaN,NaN,NaN,NaN,$23.95
40233,40235,https://store.steampowered.com/sub/354233/,sub,Valve Index Headset + Controllers,Valve Index Headset + Controllers bundle,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$749.00,NaN
40235,40237,https://store.steampowered.com/bundle/9680/Gro...,bundle,Groove Coaster - Touhou DLC Pack 04,Groove Coaster - Touhou DLC Pack 04 bundle,NaN,NaN,NaN,TAITO CORP.,NaN,...,"Single-player,Downloadable Content,Steam Achie...","English, Japanese",NaN,Action,NaN,NaN,NaN,NaN,NaN,$6.36


In [11]:
df[(df["game_description"].notna())&(df["desc_snippet"].isna())]

,index,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,...,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
826,827,https://store.steampowered.com/app/441340/Elit...,app,Elite Dangerous: Horizons Season Pass,NaN,"Mixed,(21),- 42% of the 21 user reviews in the...","Mostly Positive,(1,082),- 76% of the 1,082 use...","Dec 15, 2015",Frontier Developments,"Frontier Developments,Frontier Developments",...,"Single-player,Multi-player,MMO,Co-op,Downloada...","English,French,German,Russian,Spanish - Spain",NaN,"Action,Adventure,Massively Multiplayer,RPG,Sim...",About This Content Change the galaxy with the...,NaN,"Minimum:,OS:,Windows 7/8/10 64-bit,Processor:,...","Recommended:,OS:,Windows 7/8/10 64-bit,Process...",$29.99,$320.94
1771,1772,https://store.steampowered.com/app/57600/Tropi...,app,Tropico 3: Absolute Power,NaN,NaN,"Very Positive,(223),- 94% of the 223 user revi...","May 17, 2010",Haemimont Games,"Kalypso Media Digital,Kalypso Media Digital",...,"Single-player,Downloadable Content","English,French,German,Italian,Spanish - Spain",NaN,"Simulation,Strategy",About This Content Tropico 3: Absolute Power...,NaN,NaN,NaN,$9.99,$115.97
2104,2105,https://store.steampowered.com/app/9070/DOOM_3...,app,DOOM 3 Resurrection of Evil,NaN,NaN,"Very Positive,(338),- 84% of the 338 user revi...","Apr 3, 2005",id Software,"id Software,id Software",...,"Single-player,Multi-player,Downloadable Conten...",English,NaN,Action,About This Content The gripping expansion pa...,NaN,NaN,NaN,$2.99,$1.79
2375,2376,https://store.steampowered.com/app/201310/X3_A...,app,X3: Albion Prelude,NaN,NaN,"Very Positive,(1,042),- 92% of the 1,042 user ...","Dec 15, 2011",Egosoft,"Egosoft,Egosoft",...,"Single-player,Downloadable Content,Steam Achie...","English,French,German,Italian,Russian,Polish,S...",33.0,"Action,Simulation",About This Content X3: Albion Prelude is the ...,NaN,"Minimum:,OS:,Windows 10 / 8 / 7 / Vista SP1 / ...","Recommended:,OS:,Windows 10 / 8 / 7 / Vista SP...",$9.99,NaN
3477,3478,https://store.steampowered.com/app/8650/RACE_0...,app,RACE 07: Andy Priaulx Crowne Plaza Raceway (Fr...,NaN,NaN,"Mixed,(56),- 62% of the 56 user reviews for th...","Jun 19, 2008",SimBin,"SimBin,SimBin",...,Downloadable Content,"English,French,German,Italian,Spanish - Spain",NaN,Racing,About This Content Blue skies and white beach...,NaN,NaN,NaN,$39.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40825,40827,https://store.steampowered.com/app/899835/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,...,"Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Primo Victoria"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,NaN
40826,40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,...,"Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,NaN
40827,40829,https://store.steampowered.com/app/899832/Rock...,app,Rocksmith® 2014 Edition – Remastered – Stone T...,NaN,NaN,NaN,"Feb 5, 2019",Ubisoft - San Francisco,NaN,...,"Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,NaN
4

Columns `desc_snippet` and `game_description` has the same meaning, but I prefer `game_description` because this column has more data (words to be said) than `desc_snippet`. So I take `game_description` data, and if it is null, it will be replaced with `desc_snippet`

In [12]:
description_game = []
for i in range(len(df)):
    if df["game_description"][i] is not None:
        description_game.append(df["game_description"][i])
    elif (df["game_description"] is None)&(df["desc_snippet"]is not None):
        description_game.append(df["desc_snippet"][i])
    else:
        description_game.append("")

In [13]:
df["description_game"] = description_game
df["description_game"] = df["description_game"].fillna("")

Next, I clean up the description. There are sentences that were repeated in almost every games, these are "About This Game" and "About This Content". Also, I want the description no longer contain stopwords such as "and", "or", etc.

In [14]:
def text_format(text):
    '''
    Formatting input: change to lemma form, remove stopwords, special char, and etc.
    '''
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatizer.lemmatize(text)
    text_lemmatized = [lemmatizer.lemmatize(word) for word in text.split()]
    text = " ".join(text_lemmatized)
        
    # filter from stopwords
    stop_words = set(stopwords.words('english'))
    filtered_from_stopwords = [word for word in text.split() if not word in stop_words]
        
    text = " ".join(filtered_from_stopwords)
    text = re.sub("About This Game","",text)
    text = re.sub("About This Content","",text)
        
    return text

In [15]:
desc_clean = []
for index, row in df[['description_game']].iterrows():
    # formatting text
    outputs = text_format(row[0])
    desc_clean.append(outputs)

In [16]:
df["description_clean"]=desc_clean

## Reviews

I used `all_reviews` column to evaluate the review of the game. About the rating, I will use the predicate that steam used, such as Very Positive, Mostly Positive, etc. I got dictionary about rating predicate in steam in this discussion: https://steamcommunity.com/discussions/forum/0/1744483505466407549/

Then, I used that dictionary to be used in `all_reviews` predicate:
- Overhwelmingly Positive = 9
- Very Positive = 8
- Positive = 7
- Mostly Positive = 6
- Mixed = 5
- Mostly Negative = 4
- Negative = 3
- Very Negative = 2
- Overwhelmingly Negative = 1

The problem about this feature is the information about rating predicate and reviewers number of a games contained in a column. So I need to extract these two informations.

**Extracting Review Predicate**

In [17]:
df[['all_reviews']]

,all_reviews
0,"Very Positive,(42,550),- 92% of the 42,550 use..."
1,"Mixed,(836,608),- 49% of the 836,608 user revi..."
2,"Mostly Positive,(7,030),- 71% of the 7,030 use..."
3,"Mixed,(167,115),- 61% of the 167,115 user revi..."
4,"Mostly Positive,(11,481),- 74% of the 11,481 u..."
...,...
40826,NaN
40827,NaN
40828,NaN
40829,NaN


Notice that every rating predicate is located in the first sentence/word before the first comma. I used split function to separate this element in the list. 

In [18]:
df['all_reviews'] = df['all_reviews'].fillna("")
df['review_list'] = df['all_reviews'].str.split(',')

After I split the `all_reviews` column, this column will contain list of information about reviews separated by ",".
Now, I just need to get the first element from the list in every row, and I will get the rating predicate.

In [19]:
review_rate_str = []
for i in range(len(df)):
    review_rate_str.append(df['review_list'][i][0])

In [20]:
review_rate_str

['Very Positive',
 'Mixed',
 'Mostly Positive',
 'Mixed',
 'Mostly Positive',
 '',
 'Very Positive',
 'Very Positive',
 'Very Positive',
 'Mixed',
 'Very Positive',
 'Very Positive',
 'Mostly Positive',
 'Mixed',
 'Mostly Positive',
 '',
 'Very Positive',
 'Mostly Positive',
 'Mixed',
 'Very Positive',
 'Very Positive',
 'Very Positive',
 'Mostly Positive',
 'Very Positive',
 'Mixed',
 '',
 'Overwhelmingly Positive',
 'Very Positive',
 'Mostly Positive',
 'Very Positive',
 'Mixed',
 'Very Positive',
 'Overwhelmingly Positive',
 'Very Positive',
 'Very Positive',
 'Very Positive',
 'Mixed',
 'Very Positive',
 'Very Positive',
 '',
 '7 user reviews',
 'Positive',
 'Overwhelmingly Positive',
 'Very Positive',
 '',
 'Very Positive',
 'Very Positive',
 'Very Positive',
 'Mixed',
 'Very Positive',
 'Very Positive',
 'Mostly Positive',
 'Very Positive',
 'Very Positive',
 'Mostly Positive',
 'Mostly Positive',
 'Very Positive',
 '',
 'Overwhelmingly Positive',
 '',
 'Very Positive',
 'Mostly 

In [21]:
df["predicate_rating"] = review_rate_str

Next, I transform this predicate to integer rate according to the dictionary I used.

In [22]:
rating_int=[]
for i in range(len(df)):
    if df["predicate_rating"][i] == 'Overhwelmingly Positive':
        rating_int.append(9)
    elif df["predicate_rating"][i] == 'Very Positive':
        rating_int.append(8)
    elif df["predicate_rating"][i] == 'Positive':
        rating_int.append(7)
    elif df["predicate_rating"][i] == 'Mostly Positive':
        rating_int.append(6)
    elif df["predicate_rating"][i] == 'Mixed':
        rating_int.append(5)
    elif df["predicate_rating"][i] == 'Mostly Negative':
        rating_int.append(4)
    elif df["predicate_rating"][i] == 'Negative':
        rating_int.append(3)
    elif df["predicate_rating"][i] == 'Very Negative':
        rating_int.append(2)
    elif df["predicate_rating"][i] == 'Overhwelmingly Negative':
        rating_int.append(1)
    else:
        rating_int.append(0)

In [23]:
df["rating_int"] = rating_int

**Extracting Number of Reviewers**

In [24]:
df[['all_reviews']]

,all_reviews
0,"Very Positive,(42,550),- 92% of the 42,550 use..."
1,"Mixed,(836,608),- 49% of the 836,608 user revi..."
2,"Mostly Positive,(7,030),- 71% of the 7,030 use..."
3,"Mixed,(167,115),- 61% of the 167,115 user revi..."
4,"Mostly Positive,(11,481),- 74% of the 11,481 u..."
...,...
40826,
40827,
40828,
40829,


Every reviewer number is stored in a parentheses bracket. I just need to detect the value in this parentheses.

In [25]:
df["reviewers_number"] = df["all_reviews"].str.replace(r'[^(]*\(|\)[^)]*', '')

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [26]:
df["reviewers_number"].tolist()

['42,550',
 '836,608',
 '7,030',
 '167,115',
 '11,481',
 '',
 '9,645',
 '23,763',
 '12,127',
 '904',
 '4,600',
 '3,547',
 '14,184',
 '1,118',
 '5,484',
 '',
 '1,901',
 '1,945',
 '1,797',
 '2,442',
 '4,190',
 '382',
 '487',
 '9,757',
 '43,799',
 '',
 '40,424',
 '5,414',
 '1,094',
 '2,701',
 '15,461',
 '21,022',
 '4,612',
 '8,128',
 '5,252',
 '27,397',
 '805',
 '4,437',
 '8,950',
 '',
 '7 user reviews,- Need more user reviews to generate a score',
 '11',
 '45,829',
 '7,855',
 '',
 '7,779',
 '5,997',
 '2,396',
 '9,007',
 '417',
 '88',
 '1,731',
 '5,090',
 '28,082',
 '16,973',
 '1,786',
 '8,597',
 '',
 '561',
 '',
 '8,015',
 '5,694',
 '7,557',
 '',
 '12,784',
 '27,708',
 '',
 '16,225',
 '',
 '7,674',
 '479',
 '6,100',
 '4,030',
 '37,698',
 '',
 '12,703',
 '6,311',
 '4,809',
 '15,680',
 '35',
 '14,138',
 '',
 '37,858',
 '59,252',
 '10,495',
 '2,454',
 '',
 '2,677',
 '51',
 '54,938',
 '',
 '27,036',
 '12,948',
 '3,348',
 '7,238',
 '101,309',
 '3,511',
 '3,286',
 '8,966',
 '381',
 '9,192',
 '

Need to delete the game that not qualify enough to be predicated (too few reviews), the value such as '7 user reviews,- Need more user reviews to generate a score', '1 user reviews,- Need more user reviews to generate a score', etc.

In [27]:
df.loc[df['reviewers_number'].str.contains('user'), 'reviewers_number'] = ''

In [28]:
#replace comma with blank so that I can convert it to integer
df["reviewers_number"] = df["reviewers_number"].str.replace(',', '')

In [29]:
#replace blank reviewer or null data to 0
for i in range(len(df)):
    if df["reviewers_number"][i] == '':
        df["reviewers_number"][i] = 0
    else:
        df["reviewers_number"][i] = df["reviewers_number"][i]

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
#convert the reviewer number data to integer
df["reviewers_number"] = df["reviewers_number"].astype(dtype='int64')

In [31]:
df["reviewers_number"]

0         42550
1        836608
2          7030
3        167115
4         11481
          ...  
40826         0
40827         0
40828         0
40829         0
40830         0
Name: reviewers_number, Length: 40831, dtype: int64

Then, I used a formula from IMDb to weight the reviews based on how many reviewers in a game.
Source: https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website

In [32]:
C = df['rating_int'].mean()

In [33]:
# Function that computes the weighted rating of each games
def weighted_rating(x=df, m=10, C=C):
    v = x['reviewers_number']
    R = x['rating_int']
    # Calculation based on the formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [34]:
df['weighted_rating'] = df.apply(weighted_rating, axis=1)

In [35]:
df[['rating_int','reviewers_number','weighted_rating']].head(5)

,rating_int,reviewers_number,weighted_rating
0,8,42550,7.998742
1,5,836608,4.999972
2,6,7030,5.995236
3,5,167115,4.999859
4,6,11481,5.997081


## Popular Tags, Game Details, and Genre

In [36]:
df[['popular_tags','game_details', 'genre']]

,popular_tags,game_details,genre
0,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...",Action
1,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","Action,Adventure,Massively Multiplayer"
2,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","Action,Adventure,Strategy"
3,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","Action,Adventure,Massively Multiplayer"
4,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","Action,Free to Play,Massively Multiplayer,RPG,..."
...,...,...,...
40826,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","Casual,Simulation"
40827,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","Casual,Simulation"
40828,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...","Indie,RPG,Strategy"
40829,Action,"Single-player,Downloadable Content,Steam Achie...",Action


I want to minimize the abusive of data from a game and the originality of the data of a game. Column `popular_tags` and `genre` have similar meaning. Column `genre` is originated from the developer/publisher of the game, meanwhile `popular_tags` originated from users/players who reviewed it. I do believe that users/players who reviewed the game have the knowledge of the game which they reviewed, but to be fair for I to calculate the similarity of the games, I believe only using `genre` column is suffice enough to calculating similarity between games. So, for this project I only use `genre` column instead of `popular_tags` to identify the genre of the game. 

## Used Features

Next I made a dataframe using only columns that needed in identify similarity between games.

In [37]:
used_col = ["name", "game_details", "genre","weighted_rating","description_clean"]

In [38]:
df_fix = df[used_col]
df_fix = df_fix.fillna("")

Before finding similarity, I need to handle null data. If the `name` column is null or empty, I will exclude this game to the modelling because I can't identify which game is this. If the `description_clean` is null or empty and one of `game_details` and `genre` columns are empty or null, I will also exclude the game because I assume this game does not have enough information to find the similarity with other games.

In [39]:
null_index = []
for i in range(len(df_fix)):
    if df_fix["name"][i]=="":
        null_index.append(i)
    elif (df_fix["description_clean"][i]=="") & ((df_fix["game_details"][i]=="") or (df_fix["genre"][i]=="")):
        null_index.append(i)
    else:
        pass

In [40]:
#drop null index
df_fix = df_fix.drop(null_index)
df_fix = df_fix.reset_index()

# Modelling

## Computing Cosine Similarity

I combine the three similarity identifier to be calculated in TF-IDF Vectorizer

In [41]:
df_fix["concat"] = df_fix["description_clean"] + df_fix["game_details"] + df_fix["genre"]

In [42]:
#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_fix['concat'])

In [43]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [44]:
print(cosine_sim)

[[1.00000000e+00 4.95398031e-02 5.23689211e-02 ... 5.90949957e-02
  1.91085196e-03 8.87697748e-02]
 [4.95398031e-02 1.00000000e+00 2.95703606e-02 ... 9.73667037e-03
  1.98173450e-04 4.44145755e-02]
 [5.23689211e-02 2.95703606e-02 1.00000000e+00 ... 2.69560804e-02
  2.41814218e-03 4.67803457e-02]
 ...
 [5.90949957e-02 9.73667037e-03 2.69560804e-02 ... 1.00000000e+00
  3.41772894e-03 3.72617069e-02]
 [1.91085196e-03 1.98173450e-04 2.41814218e-03 ... 3.41772894e-03
  1.00000000e+00 4.49253866e-03]
 [8.87697748e-02 4.44145755e-02 4.67803457e-02 ... 3.72617069e-02
  4.49253866e-03 1.00000000e+00]]


In [45]:
indices_id = pd.Series(df_fix.index, index = df_fix['name']).drop_duplicates()

## Content Based Recommendation Function

In [46]:
def get_recommendations(game_name, cosine_sim=cosine_sim, df=df_fix[['name', 'weighted_rating']]):
    
    # Get the index
    idx = indices_id[game_name]
    
    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
 
    idx_list = []    
    for i in sim_scores:
        idx_game = i[0]
        idx_list.append(idx_game)
    
    # Get the scores of the 20 most similar games
    sim_scores = idx_list[1:21]
    
    # Sort based on the scores
    final_scores = df["weighted_rating"][sim_scores]
    final_scores = list(final_scores.sort_values(ascending=False).index)
    final_scores = final_scores[1:16]
    
    # get the indices
    pr_indices = [i for i in final_scores]
    
    # return the top 10 most similar programs
    return df.iloc[pr_indices]

## The Example Result

In [47]:
get_recommendations("DOOM")

,name,weighted_rating
838,Doom 3: BFG Edition,7.982834
579,DOOM 3,7.979940
1421,Victor Vran ARPG,7.979812
8639,Reflex Arena,7.957237
7679,The Haunted: Hells Reach,7.870365
7772,HordeZ,7.857986
2103,DOOM 3 Resurrection of Evil,7.846152
1650,Hell is Other Demons,6.111444
5638,RPG World - Action RPG Maker,5.915305
35749,Gone with the Demon,5.258431


In [48]:
get_recommendations("Ni no Kuni™ II: Revenant Kingdom")

,name,weighted_rating
25344,Ni no Kuni™ II: Revenant Kingdom - Season Pass,4.488278
27045,Ni no Kuni™ II: REVENANT KINGDOM - The Lair of...,4.411519
12204,Wailing Heights Complete Edition,2.646078
20279,Dogenzaka Lab Otome Games set,2.646078
6691,The Men of Yoshiwara: Kikuya and Ohgiya Bundle,2.646078
7158,Cloak and Dagger Games Collection,2.646078
37214,"How to Fool a Liar King - Insert Song ""Issho ni""",2.646078
23825,InnerSpace - Digital Deluxe,2.646078
22630,Rising Islands Collector's Edition,2.646078
12660,Wailing Heights Deluxe Edition,2.646078


In [49]:
get_recommendations("EVE Online")

,name,weighted_rating
12212,Closers: DLC Collection,2.646078
3756,Hero Siege Complete,2.646078
7154,Indie Classics By Antoni,2.646078
22556,Playito Multiplayer,2.646078
33917,Faeria Cosmetic DLC Bundle,2.646078
22470,Zombie Bundle,2.646078
2726,PRO ANGLER SPORT BUNDLE,2.646078
2109,Fishing Planet Lucky Bundle,2.646078
19136,Trading cards bundle. All in one,2.646078
7840,Space bundle,2.646078


In [50]:
get_recommendations("FINAL FANTASY X/X-2 HD Remaster")

,name,weighted_rating
413,FINAL FANTASY XIV Online,7.995100
2838,Dreamfall Chapters,7.977061
915,FINAL FANTASY III,7.949586
1035,FINAL FANTASY IV,7.941166
8798,FINAL FANTASY XV EPISODE ARDYN,7.843453
8368,The Only Traitor DLC,7.665380
883,FINAL FANTASY VI,5.967564
931,DEAD OR ALIVE 5 Last Round: Core Fighters,4.996838
1019,MOBIUS FINAL FANTASY™,4.993408
26893,FINAL FANTASY XIV: Stormblood,4.941004


The recommendation result of the algoriithm, seems good enough. But because I more prefer to find the most similar games first then I sorted by weighted rating, the games that has been recommended are seems not really poplar among gamers. especially the last 5 recommended games. So, maybe it still need an improvement.

# Notes

## Potential of Improvement
1. Try another similarity metrix out there. There are similar algorithm like cosine similarity such jaccard similarity, euclidean distance, manhattan distance, etc.
2. The data preparation process that I did is one of the rushing process because I did this project only in one day. Perhaps it could be enhanced to be better. The ones that I can think of are text cleaning process, adding data source such as number of players, popularity of the game could be added, and maybe the sorting preference can be based weighted rating first then sort by similarity.
3. This data can be given a separate ID to make it easier to identify game recommendations to be analyzed.
4. Try another null treatment to the data.

## References:
1. Steam discussion about rating: https://steamcommunity.com/discussions/forum/0/1744483505466407549/
2. IMDb Weighted Rating Formula: https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website
3. Another Content Based Recommendation Project with Same Dataset: https://www.kaggle.com/code/fetenbasak/content-based-recommendation-game-recommender
4. About Content Based Recommendation and TF-IDF: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/
5. Cosine Similarity Scikit-Learn: https://scikit-learn.org/stable/modules/metrics.html
6. Cosine Similarity Explanation: https://builtin.com/machine-learning/cosine-similarity